In [1]:
from models.inception_resnet_v1 import  InceptionResnetV1
from models.mtcnn import MTCNN
from PIL import Image,ImageDraw
import os
import cv2
import torch
import matplotlib.pyplot as plt
import numpy as np
device = 'cuda'

c:\Users\VINY\anaconda3\envs\vision\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
resnet = InceptionResnetV1(pretrained='vggface2').eval().to(device)
mtcnn = MTCNN(keep_all=True, device=device)

In [3]:
embeddings_path = './embeddings/'
embeddings = []
identities = []
for x in os.listdir(embeddings_path):
    path = os.path.join(embeddings_path, x)
    name = x.split('.')[0].split('_')[0] + ' ' + x.split('.')[0].split('_')[1]
    identities.append(name)
    embedding = torch.from_numpy(np.load(path))
    embeddings.append(embedding)
embeddings = torch.stack(embeddings,dim = 0)
embeddings.shape

torch.Size([5, 512])

In [7]:
cap = cv2.VideoCapture(0)
cap.set(cv2.CAP_PROP_FRAME_HEIGHT, 360)
cap.set(cv2.CAP_PROP_FRAME_WIDTH, 480)

while True:
    ret, img = cap.read()
    if not ret : break
    boxes, _ = mtcnn.detect(img)
    if boxes is not None:
        for box in boxes:
            x1, y1, x2, y2 = map(int, box)
            face = img[y1:y2, x1:x2]
            face = cv2.resize(face,(128,128))
            face = torch.from_numpy(face).permute(2,0,1).float()
            face = (face - 127.5)/128.0
            with torch.no_grad():
                face_embedding = resnet(face.unsqueeze(0).to(device))
                face_embedding = face_embedding.cpu().detach().squeeze(0)
            distances = torch.mean(torch.abs(embeddings - face_embedding),dim=1)
            verification_status = 'Unverified'
            if torch.min(distances) < 0.03 : 
                print(f'\r{torch.min(distances)}',end='')
                ID = torch.argmin(distances)
                verification_status = identities[ID]
            cv2.rectangle(img,(x1,y1),(x2, y2),(0,255,0),2)
            # Display "Verified" or "Unverified" above the rectangle
            cv2.putText(img, verification_status, ((x1 + x2) // 2, y1 - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 255, 255 ), 2)
            cv2.imshow('Face Verification', img)
    else:
        cv2.imshow('Face Verification', img)
    if cv2.waitKey(1) & 0xFF == ord(' '):
        break
cv2.destroyAllWindows()

KeyboardInterrupt: 

In [ ]:
face_embedding.shape

torch.Size([512])

In [ ]:
distances = torch.sum(torch.abs(embeddings - face_embedding),dim = 1) / face_embedding.numel()

In [9]:
torch.mean(torch.abs(embeddings - face_embedding),dim=1)

tensor([0.0242, 0.0385])